In [1]:
import requests
import json
import datetime
import pandas as pd

In [2]:
def r_to_j(r):
    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
#    print(myJ_s)
    myJ = json.loads(myJ_s)
#    print(json.dumps(myJ_s, indent=1))
    return myJ


In [3]:
def do_r(req, page_size=2000, page_num=0,debug=False, attrs=[]):
    myJ = {}
    rJ = {'count': 0, 'g' : {}}
    keepGoing=1
    gDebug=0

    req += '&page_size='+str(page_size)+'&page_num='

    while keepGoing:
        keepGoing=1
        page_num += 1
        if page_num > 499:
            keepGoing = 0
        
        r = requests.get(req + str(page_num) )
        
        if debug:
            print(r, page_num, datetime.datetime.now())
        
        myJ = r_to_j(r)
        
        if debug:
            print('myj=', json.dumps(myJ, indent=1))
            debug=0
        
        if len(myJ['items']) == 0:
            keepGoing=0
            continue

        rJ['count']+= len(myJ['items'])
        outJ={}
        for j in myJ['items']:
            if gDebug:
                print (json.dumps(j['umm']))
                if 'GranuleUR' in j['umm']:
                    print('****************--->', j)
                else:
                    print ('producer_granule_id NOT FOUND!', j)
                gDebug=0
            outJ={}
            for a in attrs:
                if a in j['umm']:
                    outJ[a]=j['umm'][a]
                
            rJ['g'][j['umm']['GranuleUR']]=outJ

    return rJ

In [4]:
# Get a list of short_names from a provider
import time

my_provider='ASFCLOUD'
my_shortNames=[]
#print (r.headers)
myJ_s = ''
collectionsJ={}
uniqueKeysJ={}

page_num=0
keepGoing=1
debug=1
while keepGoing:
    
    page_num += 1
    if page_num > 200:
        keepGoing = 0
    print ('before request.get')  
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?platform=AQUA&sort_key=short_name&page_size=2&page_num='+str(page_num))
    r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?provider=GES_DISC&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?instrument=MODIS&INSTRUMENT=MODIS&provider=LPDAAC_ECS&sort_key=short_name&page_size=2&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?granule_data_format=HDF-EOS2&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    print ('after request.get')  

    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
    #print(myJ_s)
    myJ = json.loads(myJ_s)
    #print(json.dumps(myJ, indent=1))

    if len(myJ['items']) == 0:
        keepGoing=0
        continue
    
    for r in myJ['items']:
        j=r['meta']
        #print ('j=', j)
        if debug == 1:
            print ('r=', json.dumps(r,indent=1))
            debug=0
        #for k in j:
        #    print ('meta',k, j[k])
        j=r['umm']
#        for k in j:
            #print ('umm', k)
#            if k == 'ShortName':
        my_shortNames.append({'ShortName' : j['ShortName'],
                                     'native-id' : r['meta']['native-id'],
                                'EntryTitle' : j['EntryTitle'],
                              'Count':r['meta']['granule-count'] if 'granule-count' in r['meta'] else 0,
                             'Format' : j ['ArchiveAndDistributionInformation'] if 'ArchiveAndDistributionInformation' in j else 'UNKNOWN'
                             })

i=1
for s in my_shortNames:
    print (i, s)
    i+=1
#print (my_shortNames)

before request.get
after request.get
r= {
 "meta": {
  "revision-id": 15,
  "deleted": false,
  "format": "application/vnd.nasa.cmr.umm+json",
  "provider-id": "GES_DISC",
  "user-id": "cloeser",
  "has-formats": false,
  "has-spatial-subsetting": false,
  "native-id": "ACOS_L2_Lite_FP_9r",
  "has-transforms": false,
  "has-variables": false,
  "concept-id": "C1720416694-GES_DISC",
  "revision-date": "2021-04-10T16:06:07.517Z",
  "granule-count": 0,
  "has-temporal-subsetting": false,
  "concept-type": "collection"
 },
 "umm": {
  "DataLanguage": "English",
  "CollectionCitations": [
   {
    "Creator": "OCO-2 Science Team/Michael Gunson, Annmarie Eldering",
    "Title": "ACOS GOSAT/TANSO-FTS Level 2 bias-corrected XCO2 and other select fields from the full-physics retrieval aggregated as daily files V9r",
    "SeriesName": "ACOS_L2_Lite_FP",
    "ReleaseDate": "2019-12-30T00:00:00.000Z",
    "ReleasePlace": "Greenbelt, MD, USA",
    "Publisher": "Goddard Earth Sciences Data and Inform

In [59]:
r_base = 'https://cmr.earthdata.nasa.gov/search/granules.umm_json?short_name='
i=1

cmr_granule_table = []

for s in my_shortNames:
   # print (s)
    if s['ShortName']=='GPM_2AGPROFNOAA20ATMS_CLIM':
        pass
    else:
        continue

    
# {'ShortName': 'AA_L2A', 
#  'native-id': 'AMSR/ADEOS-II L2A Global Swath Spatially-Resampled Brightness Temperatures V001', 
#  'EntryTitle': 'AMSR/ADEOS-II L2A Global Swath Spatially-Resampled Brightness Temperatures V001',
#  'Count': 0, 
#  'Format': {'FileDistributionInformation': [{'FormatType': 'Native', 'Format': 'HDF-EOS', 'FormatDescription': 'FTP 68 MB'}]}}
    
    r = r_base + s['ShortName']
    r += '&temporal=2019-06-01T00:00:00Z,2019-06-30T23:59:59Z'
    
    print (r)

    gJ = do_r(r, attrs=['GranuleUR','DataGranule', 'CollectionReference', 'TemporalExtent', 'SpatialExtent'],debug=False)
    
    if gJ['count'] == 0:
        continue
        
    print ('# Granules', gJ['count'])
    for g in gJ['g']:
        print ('Example G', json.dumps(gJ['g'][g]))
        break
    
    gn=0
    granuleSizeTotal=0.0
    
    print (i, 'Collection:', s['ShortName'], s['Count'], s)
    i+=1

    for g in gJ['g']:
        gn+=1
        gm=gJ['g'][g]
        #print(g, gJ['g'][g])
        #print(json.dumps(gm['granule_size']))
        #print ('x', json.dumps(gm))
        #print (gJ['g'][g]['title'])
        #if 'producer_granule_id' in gJ['g'][g]:  # Sometime its there, sometimes it isn't and I don't know why.
        #    fx = gJ['g'][g]['producer_granule_id'].split('.')[-1]
        #else:  # for asf don't use producer granule id
        
        if 'Size' in gm['DataGranule']['ArchiveAndDistributionInformation'][0]:
            pass
        else:
            gm['DataGranule']['ArchiveAndDistributionInformation'][0]['Size']='0.0'
            
        granuleSizeTotal += float(gm['DataGranule']['ArchiveAndDistributionInformation'][0]['Size'])
        
#        if gm['granule_size'] == 0.0:
#          print (gm['granule_size'])
#          gm['granule_size'] = 'NaN'
        
        if 'SpatialExtent' in gm:
            pass
        else:
            gm['SpatialExtent']=''
            
        cmr_granule_table.append ( [
            g, #GranuleUR
            gm['DataGranule'],
            gm['CollectionReference'],
            gm['TemporalExtent'],
            gm['SpatialExtent']
        ])
        
        

    if gn > 0:
        print (granuleSizeTotal)


for i in cmr_granule_table:
    print (i)
    break


https://cmr.earthdata.nasa.gov/search/granules.umm_json?short_name=GPM_2AGPROFNOAA20ATMS_CLIM&temporal=2019-06-01T00:00:00Z,2019-06-30T23:59:59Z
# Granules 427
Example G {"GranuleUR": "GPM_2AGPROFNOAA20ATMS_CLIM.05:2A-CLIM.NOAA20.ATMS.GPROF2017v2.20190531-S230053-E004221.007940.V05B.HDF5", "DataGranule": {"DayNightFlag": "Unspecified", "Identifiers": [{"Identifier": "2A-CLIM.NOAA20.ATMS.GPROF2017v2.20190531-S230053-E004221.007940.V05B.HDF5", "IdentifierType": "ProducerGranuleId"}], "ProductionDateTime": "2019-08-31T02:48:49.000Z", "ArchiveAndDistributionInformation": [{"Name": "Not provided", "Size": 5.845602989196777, "SizeUnit": "MB"}]}, "CollectionReference": {"ShortName": "GPM_2AGPROFNOAA20ATMS_CLIM", "Version": "05"}, "TemporalExtent": {"RangeDateTime": {"BeginningDateTime": "2019-05-31T23:00:53.000Z", "EndingDateTime": "2019-06-01T00:42:21.000Z"}}, "SpatialExtent": {"HorizontalSpatialDomain": {"Geometry": {"GPolygons": [{"Boundary": {"Points": [{"Longitude": -53.46243, "Latitude"

In [61]:
df_cmr=pd.DataFrame(cmr_granule_table, columns=['CollectionReference', 'GranuleUR', 'DataGranule', 'TemporalExtent', 'SpatialExtent'])
df_cmr
#df_cmr['start_ts']=pd.to_datetime(df_cmr['start'],format='%Y%m%dT%H:%M:%S.%f').dt.time
#df_cmr['end_ts']  =pd.to_datetime(df_cmr['end'],format='%Y%m%dT%H:%M:%S.%f').dt.time

#df['Open'] =       pd.to_datetime(df    ['OPEN '],format= '%H:%M:%S' ).dt.time

,CollectionReference,GranuleUR,DataGranule,TemporalExtent,SpatialExtent
0,GPM_2AGPROFNOAA20ATMS_CLIM.05:2A-CLIM.NOAA20.A...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...","{'ShortName': 'GPM_2AGPROFNOAA20ATMS_CLIM', 'V...",{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
1,GPM_2AGPROFNOAA20ATMS_CLIM.05:2A-CLIM.NOAA20.A...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...","{'ShortName': 'GPM_2AGPROFNOAA20ATMS_CLIM', 'V...",{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
2,GPM_2AGPROFNOAA20ATMS_CLIM.05:2A-CLIM.NOAA20.A...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...","{'ShortName': 'GPM_2AGPROFNOAA20ATMS_CLIM', 'V...",{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
3,GPM_2AGPROFNOAA20ATMS_CLIM.05:2A-CLIM.NOAA20.A...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...","{'ShortName': 'GPM_2AGPROFNOAA20ATMS_CLIM', 'V...",{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
4,GPM_2AGPROFNOAA20ATMS_CLIM.05:2A-CLIM.NOAA20.A...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...","{'ShortName': 'GPM_2AGPROFNOAA20ATMS_CLIM', 'V...",{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
...,...,...,...,...,...
422,GPM_2AGPROFNOAA20ATMS_CLIM.05:2A-CLIM.NOAA20.A...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...","{'ShortName': 'GPM_2AGPROFNOAA20ATMS_CLIM', 'V...",{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
423,GPM_2AGPROFNOAA20ATMS_CLIM.05:2A-CLIM.NOAA20.A...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...","{'ShortName': 'GPM_2AGPROFNOAA20ATMS_CLIM', 'V...",{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
424,GPM_2AGPROFNOAA20ATMS_CLIM.05:2A-CLIM.NOAA20.A...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...","{'ShortName': 'GPM_2AGPROFNOAA20ATMS_CLIM', 'V...",{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...
425,GPM_2AGPROFNOAA20ATMS_CLIM.05:2A-CLIM.NOAA20.A...,"{'DayNightFlag': 'Unspecified', 'Identifiers':...","{'ShortName': 'GPM_2AGPROFNOAA20ATMS_CLIM', 'V...",{'RangeDateTime': {'BeginningDateTime': '2019-...,{'HorizontalSpatialDomain': {'Geometry': {'GPo...


In [62]:
df_cmr.to_csv('cmr_spatial.tsv', sep='~', index=False)

CREATE EXTERNAL TABLE `cmr_3`(
 GranuleUR string,
 DataGranule string,
 CollectionReference string,
 TemporalExtent string,
 SpatialExtent string
)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ( 
  'quoteChar'='\u0000', 
  'separatorChar'='~') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://nasa-ems-sandbox/ems_analysis/ad-hoc/analyst1/cmr_3'
TBLPROPERTIES (
  'skip.header.line.count'='1', 
  'transient_lastDdlTime'='1620856378')

In [12]:
import sys
!{sys.executable} -m pip install PyAthena
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from pyathena import connect
from pyathena.pandas.util import to_sql

conn = connect(s3_staging_dir='s3://esdis-ems-athena', region_name='us-west-2')